In [1]:
import numpy as np
import pandas as pd
import re

import warnings 
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('car_price_prediction_updated.csv')
data.head()

,Price,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags,Age
0,13328,1399.000000,LEXUS,RX 450,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,Left wheel,Silver,12,15
1,16621,1018.000000,CHEVROLET,Equinox,Jeep,No,Petrol,3,192000,6.0,Tiptronic,4x4,Left wheel,Black,8,14
2,8467,906.838128,HONDA,FIT,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,Right-hand drive,Black,2,19
3,3607,862.000000,FORD,Escape,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,Left wheel,White,0,14
4,11726,446.000000,HONDA,FIT,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,Left wheel,Silver,4,11


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

columns_to_plot = [
    'Price', 'Mileage', 'Cylinders',
    'Airbags', 'Age', 'Engine Volume'
]

df_to_plot = data[columns_to_plot]

n_cols = 3  
n_rows = (len(columns_to_plot) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, 
                         figsize=(6 * n_cols, 5 * n_rows))
axes = axes.flatten()

for i, col_name in enumerate(columns_to_plot):
    sns.kdeplot(data=df_to_plot, x=col_name, ax=axes[i], fill=True)
    axes[i].set_title(f'Distribution of {col_name}', fontsize=12)
    axes[i].set_xlabel(col_name, fontsize=10)
    axes[i].set_ylabel('Density', fontsize=10)

for j in range(len(columns_to_plot), n_rows * n_cols):
    fig.delaxes(axes[j])
plt.tight_layout()

KeyError: "['Engine Volume'] not in index"

In [3]:
for col in data.columns:
    print(f"Column Name {col} is :\n {data[col].unique()}\n")
    print('='*35)

Column Name Price is :
 [13328 16621  8467 ... 56814 63886 22075]

Column Name Levy is :
 [ 1399.          1018.           906.83812789   862.
   446.           891.           761.           751.
   394.          1053.          1055.          1079.
   810.          2386.          1850.           531.
   586.          1249.          2455.           583.
  1537.          1288.           915.          1750.
   707.          1077.          1486.          1091.
   650.           382.          1436.          1194.
   503.          1017.          1104.           639.
   629.           919.           781.           530.
   640.           765.           777.           779.
   934.           769.           645.          1185.
  1324.           830.          1187.          1111.
   760.           642.          1604.          1095.
   966.           473.          1138.          1811.
   988.           917.          1156.           687.
 11714.           836.          1347.          2866.
  1646.  

In [5]:
# Exclude vintage vehicles (Age > 30) to ensure a homogeneous resale market.
data = data[data['Age'] <= 30]

# Apply two-sided percentile capping to mitigate the influence of extreme low and high price outliers
# while preserving the core market price distribution for robust model training.
lower_cap = data['Price'].quantile(0.02)
upper_cap = data['Price'].quantile(0.99)

data['Price'] = data['Price'].clip(lower=lower_cap, upper=upper_cap)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19045 entries, 0 to 19236
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             19045 non-null  int64  
 1   Levy              19045 non-null  float64
 2   Manufacturer      19045 non-null  object 
 3   Model             19045 non-null  object 
 4   Category          19045 non-null  object 
 5   Leather interior  19045 non-null  object 
 6   Fuel type         19045 non-null  object 
 7   Engine volume     19045 non-null  object 
 8   Mileage           19045 non-null  int64  
 9   Cylinders         19045 non-null  float64
 10  Gear box type     19045 non-null  object 
 11  Drive wheels      19045 non-null  object 
 12  Wheel             19045 non-null  object 
 13  Color             19045 non-null  object 
 14  Airbags           19045 non-null  int64  
 15  Age               19045 non-null  int64  
dtypes: float64(2), int64(4), object(10)
memory us

In [7]:
mfr_counts = data['Manufacturer'].value_counts()
mfr_to_keep = mfr_counts[mfr_counts >= 10].index
data = data[data['Manufacturer'].isin(mfr_to_keep)]

data.reset_index(drop=True, inplace=True)

In [8]:
def clean_model_name(text):
    if pd.isna(text):
        return text
    text = str(text).upper().strip()
    text = re.sub(r'[^A-Z0-9 ]', ' ', text)
    text = " ".join(text.split())
    return text

data['Model'] = data['Model'].apply(clean_model_name)

data['Model'] = data['Model'].astype(str).str.strip().str.upper()

model_counts = data['Model'].value_counts()
models_to_keep = model_counts[model_counts > 2].index

data = data[data['Model'].isin(models_to_keep)].reset_index(drop=True)

In [9]:
data['Manufacturer'] = data['Manufacturer'].str.upper().str.strip()

def clean_engine_volume(value):
    value = str(value).lower()
    is_turbo = 1 if 'turbo' in value else 0
    
    numeric_part = re.findall(r"[-+]?\d*\.\d+|\d+", value)
    volume = float(numeric_part[0]) if numeric_part else 0.0
    return volume, is_turbo

data[['Engine_Volume', 'Is_Turbo']] = data['Engine volume'].apply(
    lambda x: pd.Series(clean_engine_volume(x))
)

print("Cleaning Complete.")
print(data[['Manufacturer', 'Engine_Volume', 'Is_Turbo']].head())

Cleaning Complete.
  Manufacturer  Engine_Volume  Is_Turbo
0        LEXUS            3.5       0.0
1    CHEVROLET            3.0       0.0
2        HONDA            1.3       0.0
3         FORD            2.5       0.0
4        HONDA            1.3       0.0


In [10]:
data = data.drop(columns=['Engine volume', 'Is_Turbo', 'Levy', 'Wheel'], axis=1)

In [11]:
data.rename(columns={
    'Engine_Volume': 'Engine Volume'
}, inplace=True)

In [13]:
data

,Price,Manufacturer,Model,Category,Leather interior,Fuel type,Mileage,Cylinders,Gear box type,Drive wheels,Color,Airbags,Age,Engine Volume
0,13328,LEXUS,RX 450,Jeep,Yes,Hybrid,186005,6.0,Automatic,4x4,Silver,12,15,3.5
1,16621,CHEVROLET,EQUINOX,Jeep,No,Petrol,192000,6.0,Tiptronic,4x4,Black,8,14,3.0
2,8467,HONDA,FIT,Hatchback,No,Petrol,200000,4.0,Variator,Front,Black,2,19,1.3
3,3607,FORD,ESCAPE,Jeep,Yes,Hybrid,168966,4.0,Automatic,4x4,White,0,14,2.5
4,11726,HONDA,FIT,Hatchback,Yes,Petrol,91901,4.0,Automatic,Front,Silver,4,11,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17890,8467,MERCEDES-BENZ,CLK 200,Coupe,Yes,CNG,300000,4.0,Manual,Rear,Silver,5,26,2.0
17891,15681,HYUNDAI,SONATA,Sedan,Yes,Petrol,161600,4.0,Tiptronic,Front,Red,8,14,2.4
17892,26108,HYUNDAI,TUCSON,Jeep,Yes,Diesel,116365,4.0,Automatic,Front,Grey,4,15,2.0
17893,5331,CHEVROLET,CAPTIVA,Jeep,Yes,Diesel,51258,4.0,Automatic,Front,Black,4,18,2.0


In [21]:
max1 = data['Engine Volume'].max()
max1

np.float64(20.0)

In [14]:
X = data.drop(['Price'] , axis=1)
y = data['Price']

In [85]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

## tranformation of each column accordling for model training

one_hot = ['Category', 'Fuel type', 'Gear box type', 'Drive wheels', 'Color']

power_transform = ['Mileage', 'Age', 'Engine Volume']

standard_transform = ['Airbags']

binary_tranform = ['Leather interior']

high_cardinal = ['Manufacturer', 'Model']

## pipelines for each of the column as per need 

standard_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

power_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('power', PowerTransformer(method='yeo-johnson'))
])

one_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(
        handle_unknown='ignore',
        sparse_output=False
    ))
])

binary_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=[['No', 'Yes']], dtype=int))
])

In [86]:
## cannot be done labelencoding, onehot and ordinal for manufacture and model so using base estimator and transformer mixin for converting to numeric

from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.freq_maps = {}

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        for col in X.columns:
            self.freq_maps[col] = X[col].value_counts(normalize=True)
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        for col in X.columns:
            X[col] = X[col].map(self.freq_maps[col]).fillna(0)
        return X.values


high_card_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('freq', FrequencyEncoder())
])

In [87]:
transformer = ColumnTransformer([
    ('standard', standard_pipe, standard_transform),
    ('power', power_pipe, power_transform),
    ('onehot', one_pipe, one_hot),
    ('binary', binary_pipe, binary_tranform),
    ('high_card', high_card_pipe, high_cardinal)
], remainder='passthrough')


In [88]:
y = np.log1p(data['Price'])

In [89]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge, RidgeCV, LassoCV

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(
    X,y, test_size=0.3,
    random_state=42)

In [91]:
models = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "Ridge": Ridge(),
    "RidgeCV": RidgeCV(),
    "LassoCV": LassoCV()
}

for name, model in models.items():

    pipe = Pipeline([
        ('preprocess', transformer),
        ('model', model)
    ])
    pipe.fit(X_train, y_train)
    y_test_pred_log  = pipe.predict(X_test)

    y_test_pred  = np.expm1(y_test_pred_log)

    y_test_real  = np.expm1(y_test)

    print(name)
    print("Test  MAE:", mean_absolute_error(y_test_real, y_test_pred))
    print("Test MSE", mean_squared_error(y_test_real, y_test_pred))
    print("Test  R2 :", r2_score(y_test_real, y_test_pred))
    print("="*40)

NotFittedError: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [92]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

ensemble_transformer = ColumnTransformer([
    ('num', num_pipe, power_transform + standard_transform),
    ('onehot', one_pipe, one_hot),
    ('binary', binary_pipe, binary_tranform),
    ('high_card', high_card_pipe, high_cardinal)
], remainder='passthrough')


In [93]:
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import (
    RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor,
    GradientBoostingRegressor, HistGradientBoostingRegressor
)

from xgboost import XGBRegressor

ensemble_models = {
    'Random Forest': RandomForestRegressor(),
    'Ada Boost': AdaBoostRegressor(),
    'Extra Trees': ExtraTreesRegressor(),
    'Gradient Boost': GradientBoostingRegressor(),
    'Hist Gradient Boost': HistGradientBoostingRegressor(),
    'XG Boost': XGBRegressor()
}

In [94]:
for name, model in ensemble_models.items():

    pipe = Pipeline([
        ('preprocess', ensemble_transformer),
        ('model', model)
    ])
    pipe.fit(X_train, y_train)
    y_test_pred_log  = pipe.predict(X_test)

    y_test_pred  = np.expm1(y_test_pred_log)

    y_test_real  = np.expm1(y_test)

    print(name)
    print("Test  MAE:", mean_absolute_error(y_test_real, y_test_pred))
    print("Test  R2 :", r2_score(y_test_real, y_test_pred))
    print("="*40)

NotFittedError: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [22]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, uniform
svr_pipe = Pipeline([
    ('preprocess', transformer),
    ('model', SVR(kernel='rbf'))
])

svr_param_dist = {
    'model__C': loguniform(1e-1, 1e3),
    'model__gamma': loguniform(1e-3, 1e-1),
    'model__epsilon': uniform(0.01, 0.2)
}
svr_random = RandomizedSearchCV(
    svr_pipe,
    svr_param_dist,
    n_iter=30,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42
)

svr_random.fit(X_train, y_train)

KeyboardInterrupt: 

In [43]:
rf_pipe = Pipeline([
    ('preprocess', ensemble_transformer),
    ('model', RandomForestRegressor(
        random_state=42,
        n_jobs=-1
    ))
])
from sklearn.model_selection import RandomizedSearchCV

rf_param_dist = {
    'model__n_estimators': [200, 400, 600, 800],
    'model__max_depth': [None, 10, 15, 20, 25],
    'model__min_samples_leaf': [1, 3, 5, 10],
    'model__max_features': ['sqrt', 0.5, 0.7]
}

rf_random = RandomizedSearchCV(
    rf_pipe,
    rf_param_dist,
    n_iter=25,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42
)

rf_random.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocess',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['Levy',
                                                                                'Mileage',
                                                                                'Cylinders',
                                                                                'Age',
                                                                                'Engine '
                                                                                'Volume',
                                                                                'Airbags']),
                                                                              ('onehot',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('onehot',
                                                                                                OneHotE...
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('freq',
                                                                                                FrequencyEncoder())]),
                                                                               ['Manufacturer',
                                                                                'Model'])])),
                                             ('model',
                                              RandomForestRegressor(n_jobs=-1,
                                                                    random_state=42))]),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'model__max_depth': [None, 10, 15, 20,
                                                             25],
                                        'model__max_features': ['sqrt', 0.5,
                                                                0.7],
                                        'model__min_samples_leaf': [1, 3, 5,
                                                                    10],
                                        'model__n_estimators': [200, 400, 600,
                                                                800]},
                   random_state=42, scoring='r2')

In [42]:
et_pipe = Pipeline([
    ('preprocess', ensemble_transformer),
    ('model', ExtraTreesRegressor(
        random_state=42,
        n_jobs=-1
    ))
])

et_param_dist = {
    'model__n_estimators': [300, 500, 700, 900],
    'model__max_depth': [None, 15, 20, 25],
    'model__min_samples_leaf': [1, 3, 5],
    'model__max_features': ['sqrt', 0.5, 0.7]
}

et_random = RandomizedSearchCV(
    et_pipe,
    et_param_dist,
    n_iter=25,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42
)

et_random.fit(X_train, y_train)

NameError: name 'RandomizedSearchCV' is not defined

In [ ]:
hyper_tune_models = {
    'SVR': svr_random,
    'RandomForest': rf_random,
    'ExtraTrees': et_random
}

for name, rs in hyper_tune_models.items():
    print(name)
    print("Best CV R2:", rs.best_score_)
    print("Best Params:", rs.best_params_)
    print("="*40)

SVR
Best CV R2: 0.43959983808100445
Best Params: {'model__C': 15.375920235481747, 'model__epsilon': 0.04697089110510541, 'model__gamma': 0.08692991511139551}
RandomForest
Best CV R2: 0.6390123263595453
Best Params: {'model__n_estimators': 800, 'model__min_samples_leaf': 1, 'model__max_features': 0.5, 'model__max_depth': None}
ExtraTrees
Best CV R2: 0.6118763923426973
Best Params: {'model__n_estimators': 300, 'model__min_samples_leaf': 1, 'model__max_features': 0.7, 'model__max_depth': 25}


In [44]:
final_rf = Pipeline([
    ('preprocess', ensemble_transformer),
    ('model', RandomForestRegressor(
        n_estimators=800,
        max_depth=None,
        min_samples_leaf=1,
        max_features=0.5,
        random_state=42,
        n_jobs=-1
    ))
])

from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(
    final_rf,
    X,
    y,
    cv=5,
    scoring='r2',
    n_jobs=-1
)

print("CV R2 scores:", cv_scores)
print("Mean CV R2   :", np.mean(cv_scores))
print("Std CV R2    :", np.std(cv_scores))

ValueError: Found input variables with inconsistent numbers of samples: [17602, 15884]

In [ ]:
import joblib
joblib.dump(final_rf, "random_forest_car_price_model.pkl")

['random_forest_car_price_model.pkl']